In [5]:
%load_ext autoreload
%autoreload 3
%reload_ext autoreload
from IPython.core import ultratb

ultratb.VerboseTB._tb_highlight = "bg:ansired"

from common import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
raw_data_dir_path = Path("/home/cecilvega/Public/MEL")
dirs = [d for d in raw_data_dir_path.rglob("*") if d.is_dir()]

files = [f for f in raw_data_dir_path.rglob("*") if ((f.is_file()) & (f.suffix == ".zip"))]

df_dirs = pd.DataFrame(dirs, columns=["dir_path"]).assign(dir_stem=lambda x: x["dir_path"].map(lambda x: x.stem))
df_tar = pd.DataFrame(files, columns=["path"]).assign(
    stem=lambda x: x["path"].map(lambda x: x.stem), dir_path=lambda x: x["path"].map(lambda x: x.parent / Path(x.stem))
)
df_tar = df_tar.merge(df_dirs, how="left", on="dir_path")
df_tar = df_tar.assign(parent_dir_path=df_tar["path"].map(lambda x: x.parent))

df_tar

,path,stem,dir_path,dir_stem,parent_dir_path
0,/home/cecilvega/Public/MEL/TK875/Data_875_ener...,Data_875_enero 2023,/home/cecilvega/Public/MEL/TK875/Data_875_ener...,Data_875_enero 2023,/home/cecilvega/Public/MEL/TK875
1,/home/cecilvega/Public/MEL/TK851/851_psc_20220...,851_psc_20220829-095306,/home/cecilvega/Public/MEL/TK851/851_psc_20220...,851_psc_20220829-095306,/home/cecilvega/Public/MEL/TK851
2,/home/cecilvega/Public/MEL/TK320/2022-09-02 TK...,2022-09-02 TK320,/home/cecilvega/Public/MEL/TK320/2022-09-02 TK320,2022-09-02 TK320,/home/cecilvega/Public/MEL/TK320
3,/home/cecilvega/Public/MEL/TK863/863_dsc_20221...,863_dsc_20221231-102525,/home/cecilvega/Public/MEL/TK863/863_dsc_20221...,863_dsc_20221231-102525,/home/cecilvega/Public/MEL/TK863
4,/home/cecilvega/Public/MEL/TK882/Data_882.zip,Data_882,/home/cecilvega/Public/MEL/TK882/Data_882,Data_882,/home/cecilvega/Public/MEL/TK882
...,...,...,...,...,...
662,/home/cecilvega/Public/MEL/TK867/2023-12-14 TK...,KMS_Export,/home/cecilvega/Public/MEL/TK867/2023-12-14 TK...,NaN,/home/cecilvega/Public/MEL/TK867/2023-12-14 TK...
663,/home/cecilvega/Public/MEL/TK867/2022-08-22 TK...,KMS_Export,/home/cecilvega/Public/MEL/TK867/2022-08-22 TK...,NaN,/home/cecilvega/Public/MEL/TK867/2022-08-22 TK...
664,/home/cecilvega/Public/MEL/TK867/2023-06-24 TK...,KMS_Export,/home/cecilvega/Public/MEL/TK867/2023-06-24 TK...,NaN,/home/cecilvega/Public/MEL/TK867/2023-06-24 TK...
665,/home/cecilvega/Public/MEL/TK867/2023-10-19 TK...,KMS_Export,/home/cecilvega/Public/MEL/TK867/2023-10-19 TK...,NaN,/home/cecilvega/Public/MEL/TK867/2023-10-19 TK...


In [ ]:
files_dir = Path("/home/cecilvega/Public/MEL")
files = [
    str(f)
    for f in files_dir.rglob("*")
    if (
        (f.is_file())
        & ("~lock" not in f.name)
        & (f.suffix in [".csv", ".db"])
        & (
            (
                any(
                    s in f.stem
                    for s in [
                        "LR",
                        "egdata",
                        "haulcycle",
                        "trqswdata",
                        "tripdata",
                        # "Haul"
                        # "dp_",
                        # "fault0",
                    ]
                )
            )
            | (f.stem == "events")
            | (("events_en" in f.stem) & ("engr" not in f.stem))
            | (f.name == "Export_Haul.db")
            | (("MINE" in f.stem) & (any(s in f.stem for s in ["SERIAL_", "PROFILE_", "COUNTER_"])))
        )
    )
]
files = pl.DataFrame({"raw_path": files}).with_columns(
    file_size=pl.col("raw_path").apply(lambda x: Path(x).stat().st_size)
)

asset_key_map = {"plm": {"stem_pattern": "Haul"}}